In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install Qiskit
!{sys.executable} -m pip install qiskit_ibm_runtime
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pylatexenc
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install qiskit_aer

## Transpiling a Circuit

The high-level gates used to implement quantum algorithms cannot always be directly applied on real hardware. Real quantum devices support only a limited set of native basis gates, which vary depending on the hardware. 

To execute a circuit on a specific device, it must be transpiled into an equivalent circuit using the native gates of the hardware.

Fortunately, the Solovay-Kitaev theorem ensures that any quantum gate can be approximated using a finite set of single-qubit gates:

$
\text{Solovay-Kitaev Theorem: A set of single-qubit gates can approximate any desired quantum gate with a short sequence of gates.}
$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, QuantumRegister
from qiskit import *
from qiskit.primitives import StatevectorSampler
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService

Initializing some variables

In [ ]:
Number_of_qubits = 5
Total_number_of_cells = 2**Number_of_qubits
density = np.linspace(0, 1, Total_number_of_cells)
density_normalized = density / np.linalg.norm(density)

Initialize a quantum circuit and the initialization method.

In [ ]:
qx = QuantumRegister(Number_of_qubits,'qx')
qc = QuantumCircuit(qx)
qc.initialize(density_normalized, qx)

The $\texttt{transpile}$ function rewrites a quantum circuit to match the native gate set of the target hardware while optimizing its structure. For example:

$
\texttt{transpiled\_circuit = transpile(qc, basis\_gates=['rz', 'sx', 'x', 'cx'], optimization\_level=3)}.
$

This converts the circuit $\texttt{qc}$ into an equivalent form using the specified basis gates ($\texttt{rz}$, $\texttt{sx}$, $\texttt{x}$, and $\texttt{cx}$) and applies optimizations to reduce gate count and circuit depth. The $\texttt{optimization\_level=3}$ ensures maximum optimization, improving execution fidelity on noisy quantum devices.


In [ ]:
transpiled_circuit = transpile(qc, basis_gates=['rz', 'sx', 'x', 'cx'], optimization_level=3)

In [ ]:
qc.draw(output='mpl')

In [ ]:
transpiled_circuit.draw(output='mpl')

To analyze the transpiled circuit:
- $\texttt{transpiled\_circuit.count\_ops()}$ returns the number of each gate type.
- The total number of gates is computed as:
  
  $
  \texttt{total\_gates = sum(gate\_counts.values())}.
  $

- The circuit depth, indicating the longest sequence of operations, is obtained using:
  
  $
  \texttt{transpiled\_circuit.depth()}.
  $
  
These metrics summarize the circuit's complexity and resource usage.


In [ ]:
gate_counts = transpiled_circuit.count_ops()
print("Gate counts:", gate_counts)
total_gates = sum(gate_counts.values())
print("Total number of gates:", total_gates)
circuit_depth = transpiled_circuit.depth()
print("Circuit depth:", circuit_depth)


Now compute Total number of gates and the circuit depth for a varying number of qubits.

In [ ]:
#Range from 1 to 12 qubits
Number_of_qubits_for_loop = 12
total_gates = np.zeros(Number_of_qubits_for_loop)
circuit_depth = np.zeros(Number_of_qubits_for_loop)


for qubit_number in range(1,Number_of_qubits_for_loop):
    Total_number_of_cells = 2**qubit_number
    density = np.random.rand(Total_number_of_cells)
    #density = np.linspace(0, 1, Total_number_of_cells)
    density_normalized = density / np.linalg.norm(density)
    qx = QuantumRegister(qubit_number,'qx')
    qc = QuantumCircuit(qx)
    qc.initialize(density_normalized, qx)
    transpiled_circuit = transpile(qc, basis_gates=['rz', 'sx', 'x', 'cx'], optimization_level=3)

    gate_counts = transpiled_circuit.count_ops()

    total_gates[qubit_number] = sum(gate_counts.values())
    circuit_depth[qubit_number] = transpiled_circuit.depth()


Note: The y-axis is scaled logarithmically.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

ax1.plot(total_gates, marker="o", linestyle="None")
ax1.set_yscale('log')
ax1.set_xlabel("qubit number")
ax1.set_ylabel("total gates")
ax1.set_title("Transpiled circuit gate count")
ax1.grid(True)

ax2.plot(circuit_depth, marker="o", linestyle="None")
ax2.set_yscale('log')
ax2.set_xlabel("qubit number")
ax2.set_ylabel("circuit depth")
ax2.set_title("Transpiled circuit depth")
ax2.grid(True)

plt.show()

Now we do the same for the streaming circuit implemented in the second notebook.
We compare the computational complexity of the two approaches for varying qubit number.

In [ ]:
from qiskit.circuit.library import QFT
from numpy import pi
#Range from 1 to 6 qubits
Number_of_qubits_for_loop = 12
total_gates_cx = np.zeros(Number_of_qubits_for_loop)
circuit_depth_cx = np.zeros(Number_of_qubits_for_loop)
total_gates_qft = np.zeros(Number_of_qubits_for_loop)
circuit_depth_qft = np.zeros(Number_of_qubits_for_loop)

for qubit_number in range(1,Number_of_qubits_for_loop):
    qx = QuantumRegister(qubit_number,'qx')
    qc = QuantumCircuit(qx)
    qc.x(0)
    for ii in range(1,qubit_number):
        qc.mcx(qx[:ii],qx[ii])
    transpiled_circuit = transpile(qc, basis_gates=['rz', 'sx', 'x', 'cx'], optimization_level=3)

    gate_counts = transpiled_circuit.count_ops()

    total_gates_cx[qubit_number] = sum(gate_counts.values())
    circuit_depth_cx[qubit_number] = transpiled_circuit.depth()


    #QFT circuit
    qx = QuantumRegister(qubit_number,'qx')
    qc = QuantumCircuit(qx)
    qft_circuit = QFT(qubit_number, do_swaps=True, inverse=False, approximation_degree=0)
    qft_inverse_circuit = QFT(qubit_number, do_swaps=True, inverse=True, approximation_degree=0)

    #Positive streaming
    qc.append(qft_circuit, qx[:])
    for ii in range(len(qx)):
        theta = 2**(ii+1)*pi/(2**(len(qx)))
        qc.p(theta,qx[ii])
    qc.append(qft_inverse_circuit, qx[:])


    transpiled_circuit = transpile(qc, basis_gates=['rz', 'sx', 'x', 'cx'], optimization_level=3)

    gate_counts = transpiled_circuit.count_ops()

    total_gates_qft[qubit_number] = sum(gate_counts.values())
    circuit_depth_qft[qubit_number] = transpiled_circuit.depth()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

ax1.plot(total_gates_cx, marker="o", linestyle="None",label='canonical shift')
ax1.plot(total_gates_qft, marker="o", linestyle="None",label='qft shift')
ax1.legend()
ax1.set_yscale('log')
ax1.set_xlabel("qubit number")
ax1.set_ylabel("total gates")
ax1.set_title("Transpiled circuit gate count")
ax1.grid(True)

ax2.plot(circuit_depth_cx, marker="o", linestyle="None",label='canonical shift')
ax2.plot(circuit_depth_qft, marker="o", linestyle="None",label='qft shift')
ax2.legend()
ax2.set_yscale('log')
ax2.set_xlabel("qubit number")
ax2.set_ylabel("circuit depth")
ax2.set_title("Transpiled circuit depth")
ax2.grid(True)

plt.show()

## Connectivity of qubits
Real quantum hardware is constrained by both basis gates and qubit connectivity. Unlike previously assumed fully connected quantum computer, real devices, such as IBM's Kawasaki with 127 qubits, have limited connectivity. During transpilation, the circuit must account for these constraints to ensure operations comply with the hardware's coupling map.


In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeKawasaki
from qiskit.visualization import plot_gate_map
backend = FakeKawasaki()
print(
    f"Name: {backend.name}\n"
    f"Version: {backend.version}\n"
    f"No. of qubits: {backend.num_qubits}\n"
)

In [ ]:
# #graphviz needs to be installed on the computer
# plot_gate_map(
#     backend,
#     plot_directed=True,
# )

Here we apply the canonical and qft streaming operator on two different quantum circuits and consider in the transpilation the qubit connectivity and the native gate set.
In the loop we vary the number of qubits on which the operators are applied to.

In [ ]:
from qiskit.circuit.library import QFT
from numpy import pi
#Range from 1 to 6 qubits
Number_of_qubits_for_loop = 12
total_gates_cx_real_hardware = np.zeros(Number_of_qubits_for_loop)
circuit_depth_cx_real_hardware = np.zeros(Number_of_qubits_for_loop)
total_gates_qft_real_hardware = np.zeros(Number_of_qubits_for_loop)
circuit_depth_qft_real_hardware = np.zeros(Number_of_qubits_for_loop)

for qubit_number in range(1,Number_of_qubits_for_loop):
    qx = QuantumRegister(qubit_number,'qx')
    qc = QuantumCircuit(qx)
    qc.x(0)
    for ii in range(1,qubit_number):
        qc.mcx(qx[:ii],qx[ii])
    transpiled_circuit = transpile(qc, backend=backend, optimization_level=3)

    gate_counts = transpiled_circuit.count_ops()

    total_gates_cx_real_hardware[qubit_number] = sum(gate_counts.values())
    circuit_depth_cx_real_hardware[qubit_number] = transpiled_circuit.depth()


    #QFT circuit
    qx = QuantumRegister(qubit_number,'qx')
    qc = QuantumCircuit(qx)
    qft_circuit = QFT(qubit_number, do_swaps=True, inverse=False, approximation_degree=0)
    qft_inverse_circuit = QFT(qubit_number, do_swaps=True, inverse=True, approximation_degree=0)

    #Positive streaming
    qc.append(qft_circuit, qx[:])
    for ii in range(len(qx)):
        theta = 2**(ii+1)*pi/(2**(len(qx)))
        qc.p(theta,qx[ii])
    qc.append(qft_inverse_circuit, qx[:])


    transpiled_circuit = transpile(qc, backend=backend, optimization_level=3)

    gate_counts = transpiled_circuit.count_ops()

    total_gates_qft_real_hardware[qubit_number] = sum(gate_counts.values())
    circuit_depth_qft_real_hardware[qubit_number] = transpiled_circuit.depth()

We plot the total number of gates and the circuit depth for both streaming operators. The results are compared for two cases:
- **Blue markers**: Transpilation considering qubit connectivity.
- **Red markers**: Transpilation without considering qubit connectivity.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

ax1.plot(total_gates_cx, marker="o",color='red', linestyle="None",label='canonical shift')
ax1.plot(total_gates_cx_real_hardware,color='blue', marker="o", linestyle="None",label='canonical shift hardware')
ax1.plot(total_gates_qft, marker="x",color='red', linestyle="None",label='qft shift')
ax1.plot(total_gates_qft_real_hardware,color='blue', marker="x", linestyle="None",label='qft shift hardware')
ax1.legend()
ax1.set_yscale('log')
ax1.set_xlabel("qubit number")
ax1.set_ylabel("total gates")
ax1.set_title("Transpiled circuit gate count")
ax1.grid(True)

ax2.plot(circuit_depth_cx, color='red', marker="o", linestyle="None",label='canonical shift')
ax2.plot(circuit_depth_cx_real_hardware, color='blue', marker="o", linestyle="None",label='canonical shift hardware')
ax2.plot(circuit_depth_qft, color='red', marker="x", linestyle="None",label='qft shift')
ax2.plot(circuit_depth_qft_real_hardware, color='blue' , marker="x", linestyle="None",label='qft shift hardware')
ax2.legend()
ax2.set_yscale('log')
ax2.set_xlabel("qubit number")
ax2.set_ylabel("circuit depth")
ax2.set_title("Transpiled circuit depth")
ax2.grid(True)

plt.show()